In [1]:
from langchain.prompts import  PromptTemplate
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatGooglePalm
from langchain import LLMChain
from langchain.llms import GooglePalm
import re
from key import google_api_key
from datetime import datetime
import pandas as pd


pattern = r'[^\w\s]'
    

search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]



In [38]:
class careerNinjaModel:
    def __init__(self):
        #Model declaration to chat with the user

        self.username = None
        self.llm_chat = ChatGooglePalm(google_api_key = google_api_key)
     
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.careerNinjaModel_llm  = initialize_agent(tools,
                               self.llm_chat,
                               agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
                                verbose=False,
                            handle_parsing_errors=True,
                              memory=self.memory)

        #Model declaration to check the tone of the sentences
        self.prompt_template  = PromptTemplate(
                input_variables=['topic'],
                template="Please analyze the tone of content related to '{topic}' and classify it as either friendly or non-friendly.no additional information" )
        self.check_tone_model  =LLMChain(prompt=self.prompt_template,llm=GooglePalm(google_api_key = google_api_key))
        self.sentence = None
        self.user_tone = None

        
        #Model declaration to collect user information
        self.data_prompt_template = PromptTemplate(
                input_variables=['user_deatils'],
                template="Ask the user politely and in a innovative way about there {user_deatils} in English only one question in first person" )
        self.user_data_model  =LLMChain(prompt=self.data_prompt_template,llm=ChatGooglePalm(google_api_key = google_api_key))
        self.verification = None
        

    def check_tone(self):
        try:
            predection = self.check_tone_model.run(topic = self.sentence)
            if predection == "friendly":
                self.user_tone = True
            else:
                self.user_tone = False      
        except Exception as e:
             self.user_tone = False

    
    def user_promt(self):
        ans = self.careerNinjaModel_llm(self.sentence)
        if self.username != None:
            return "Hello   " +self.username + "  Information about  " + ans['output']
        return ans['output']

    
    def ask_user(self,detail):
    
        question = self.user_data_model.run(user_deatils = detail)
        return question

    
    def check_user_details(self,text,key):
        data_prompt_template = PromptTemplate(
                    input_variables=['key','text'],
                    template="Is this {text} valid {key} just give the answer as yes or no" )
        
        user_model_check = LLMChain(prompt=data_prompt_template,llm=GooglePalm(google_api_key = google_api_key))
        try:
            predection = user_model_check.run(key = key,text = text)
            if predection == "yes":
                self.verification = True
            else:
                self.verification = False
        except Exception as e:
            self.verification = False

    from datetime import datetime

    def validate_birthday(self,birthday):
        try:
            birthdate = datetime.strptime(birthday, '%Y-%m-%d')
            

            current_date = datetime.now()
            
            age = current_date.year - birthdate.year - ((current_date.month, current_date.day) < (birthdate.month, birthdate.day))
            if age < 150 and birthdate <= current_date:
                return True,
            else:
                return False
        except ValueError:
            return False
            
    def validate_phone_number(self,phone_number):

            pattern = r'^\d{10}$'
            if re.match(pattern, phone_number):
                return True
            else:
                return False
                
    def validate_email(self,email):
 
            pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
            if re.match(pattern, email):
                return True
            else:
                return False

    def save_user_data(self,user_data):
        df = pd.DataFrame([user_data])
        csv_file = 'data.csv'
        df.to_csv(csv_file, index=False)

                
            
        
        
model = careerNinjaModel()      

In [39]:
user_data = {"name":None,"birthday":None,"Address":None,"Phone-Number":None,"Email":None}

print("Hi How are you what would you like to know about ???   enter exit or close to exit anytime")
while True:
    if user_data['name']!=None:
                    model.username = user_data['name']
    print("----------------------------------------------------------------------------------------------------------------------")
    user_input = str(input())
    if user_input.lower() == "exit":
        model.save_user_data(user_data)
        break
    print("----------------------------------------------------------------------------------------------------------------------")
    model.sentence = user_input
    ans = model.user_promt()
    print("----------------------------------------------------------------------------------------------------------------------")
    print("Chatbot response : ", ans)
    print("----------------------------------------------------------------------------------------------------------------------")
    model.check_tone()
    if model.user_tone:
        for key, value in user_data.items():
            if value is None:
     
                question = model.ask_user(key)
                result_string = re.sub(pattern, '', question)
                print("----------------------------------------------------------------------------------------------------------------------")
                print("Chatbot response:",result_string)
                print("----------------------------------------------------------------------------------------------------------------------")
                user_response = input()
                
                if key ==  'name':
                    print("----------------------------------------------------------------------------------------------------------------------")
                    print("Chatbot response: Nice to mee you",user_response)
                    print("----------------------------------------------------------------------------------------------------------------------")
                if key == 'birthday':
                    predection = model.validate_birthday(user_response)
                    if predection:
                        user_data['birthday'] = user_response
                    else:
                        print("----------------------------------------------------------------------------------------------------------------------")
                        print("Chatbot response : No problem lets continue with our Topic today")
                        print("----------------------------------------------------------------------------------------------------------------------")
                        break
                elif key == 'Address':
                  
                        user_data['Address'] = user_response
        
                elif key == 'name':
                   
                        user_data['name'] = user_response
                
                elif key == 'Email':
                    if model.validate_email(user_response):
                        user_data['Email'] = user_response
                    else:
                        print("----------------------------------------------------------------------------------------------------------------------")
                        print("Chatbot response : No problem lets continue with our Topic today")
                        print("----------------------------------------------------------------------------------------------------------------------")
                        break
                elif key == 'Phone-Number':
                    if model.validate_phone_number(user_response):
                        user_data['Phone-Number'] = user_response
                    else:
                        print("----------------------------------------------------------------------------------------------------------------------")
                        print("Chatbot response : No problem lets continue with our Topic today")
                        print("----------------------------------------------------------------------------------------------------------------------")
                        break
                    
                


      

Hi How are you what would you like to know about ???   enter exit or close to exit anytime
----------------------------------------------------------------------------------------------------------------------


 hiiii how are you


----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
Chatbot response :  I am doing well, thank you for asking! How are you doing today?
----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
Chatbot response: Hello there Im glad to meet you May I know your name
----------------------------------------------------------------------------------------------------------------------


 sahil


----------------------------------------------------------------------------------------------------------------------
Chatbot response: Nice to mee you sahil
----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
Chatbot response: Hi there I hope youre doing well Im just curious when is your birthday Id love to wish you a happy birthday in advance
----------------------------------------------------------------------------------------------------------------------


...

 jainsahil1116@gmail.com


----------------------------------------------------------------------------------------------------------------------


 anime


----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
Chatbot response :  Hello   sahil  Information about  Anime is a term used to describe all forms of animation from Japan. It is a diverse medium that encompasses a wide range of genres, styles, and techniques. Anime has been around for over a century, and it has become increasingly popular in recent years. There are many reasons for the popularity of anime, including its unique art style, its complex characters, and its often thought-provoking stories. Anime can be enjoyed by people of all ages, and it is a great way to learn about Japanese culture. If you are interested in watching anime, there are many resources available online and in stores. You can also find anime at many local libraries and community centers. So what are you waiting for? Start wa

 exit


{'name': 'sahil',
 'birthday': '2001-11-16',
 'Address': 'mysore',
 'Phone-Number': '9980670768',
 'Email': 'jainsahil1116@gmail.com'}